In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Загрузка данных
data = pd.read_csv('bank-full.csv', sep=';')

selected_features = ['age', 'job', 'marital', 'education', 'balance', 'housing',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
                    'previous', 'poutcome', 'y']
dataset = data[selected_features].copy()

# Проверяем наличие пропущенных значений
print("Анализ пропущенных значений:")
missing_analysis = dataset.isnull().sum()
print(missing_analysis[missing_analysis > 0])
print("Пропущенных значений не обнаружено\n")

# ВОПРОС 1

education_counts = dataset['education'].value_counts()
most_frequent_education = education_counts.index[0]
print(f"ВОПРОС 1 - Самое частое значение education: {most_frequent_education}")

# ВОПРОС 2

numerical_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
correlation_analysis = dataset[numerical_features].corr()

np.fill_diagonal(correlation_analysis.values, 0)
max_corr_value = correlation_analysis.abs().max().max()
max_corr_pair = [(i, j) for i in correlation_analysis.columns
                 for j in correlation_analysis.columns
                 if abs(correlation_analysis.loc[i, j]) == max_corr_value][0]

print(f"\nВОПРОС 2 - Наибольшая корреляция: {max_corr_pair[0]} и {max_corr_pair[1]} ({max_corr_value:.3f})")

# Преобразование целевой переменной
dataset['target'] = dataset['y'].apply(lambda x: 1 if x == 'yes' else 0)
features_data = dataset.drop(['y', 'target'], axis=1)
target_data = dataset['target']

# Разделение на обучающую, валидационную и тестовую выборки
X_main, X_eval, y_main, y_eval = train_test_split(features_data, target_data,
                                                   test_size=0.4, random_state=42,
                                                   stratify=target_data)
X_val, X_test, y_val, y_test = train_test_split(X_eval, y_eval,
                                                test_size=0.5, random_state=42,
                                                stratify=y_eval)

print(f"\nРазмеры выборок:")
print(f"Обучающая: {X_main.shape[0]}")
print(f"Валидационная: {X_val.shape[0]}")
print(f"Тестовая: {X_test.shape[0]}")

# ВОПРОС 3

categorical_vars = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

# Подготовка данных
X_train_cat = X_main[categorical_vars].copy()
for col in categorical_vars:
    X_train_cat[col] = X_train_cat[col].astype('category').cat.codes

mi_scores = mutual_info_classif(X_train_cat, y_main, random_state=42)
mi_df = pd.DataFrame({'Признак': categorical_vars, 'MI_Score': mi_scores})
mi_df['MI_Score'] = mi_df['MI_Score'].round(2)
mi_df = mi_df.sort_values('MI_Score', ascending=False)

print("\nВОПРОС 3 - Mutual Information анализ:")
print(mi_df)
highest_mi_feature = mi_df.iloc[0]['Признак']
print(f"Признак с наибольшей взаимной информацией: {highest_mi_feature}")

# Определяем типы признаков
categorical_cols = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Создаем препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ])

# ВОПРОС 4
# Создаем и обучаем модель
base_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
])

base_model.fit(X_main, y_main)
val_predictions = base_model.predict(X_val)
baseline_accuracy = accuracy_score(y_val, val_predictions)

print(f"\nВОПРОС 4 - Точность базовой модели: {baseline_accuracy:.2f}")

# ВОПРОС 5
features_to_remove = ['age', 'balance', 'marital', 'previous']
accuracy_changes = {}

print("\nВОПРОС 5 - Анализ важности признаков:")
for feature in features_to_remove:
    # Удаляем признак из данных
    X_main_reduced = X_main.drop(columns=[feature])
    X_val_reduced = X_val.drop(columns=[feature])

    # Обновляем препроцессор
    if feature in numerical_cols:
        reduced_num_cols = [col for col in numerical_cols if col != feature]
        reduced_cat_cols = categorical_cols.copy()
    else:
        reduced_num_cols = numerical_cols.copy()
        reduced_cat_cols = [col for col in categorical_cols if col != feature]

    reduced_preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), reduced_num_cols),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), reduced_cat_cols)
        ])

    reduced_model = Pipeline(steps=[
        ('preprocessor', reduced_preprocessor),
        ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
    ])

    reduced_model.fit(X_main_reduced, y_main)
    reduced_predictions = reduced_model.predict(X_val_reduced)
    reduced_accuracy = accuracy_score(y_val, reduced_predictions)

    accuracy_diff = baseline_accuracy - reduced_accuracy
    accuracy_changes[feature] = abs(accuracy_diff)
    print(f"Без '{feature}': точность = {reduced_accuracy:.3f}, изменение = {accuracy_diff:.3f}")

least_important = min(accuracy_changes, key=accuracy_changes.get)
print(f"Наименее важный признак: {least_important}")


# ВОПРОС 6

regularization_params = [0.01, 0.1, 1, 10]
best_reg_accuracy = 0
optimal_C = None

print("\nВОПРОС 6 - Подбор параметра регуляризации:")
for C_param in regularization_params:
    reg_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(solver='liblinear', C=C_param, max_iter=1000, random_state=42))
    ])

    reg_model.fit(X_main, y_main)
    reg_predictions = reg_model.predict(X_val)
    reg_accuracy = accuracy_score(y_val, reg_predictions)

    print(f"C = {C_param}: точность = {reg_accuracy:.3f}")

    if reg_accuracy > best_reg_accuracy:
        best_reg_accuracy = reg_accuracy
        optimal_C = C_param

print(f"Оптимальное значение C: {optimal_C}")

Анализ пропущенных значений:
Series([], dtype: int64)
Пропущенных значений не обнаружено

ВОПРОС 1 - Самое частое значение education: secondary

ВОПРОС 2 - Наибольшая корреляция: pdays и previous (0.455)

Размеры выборок:
Обучающая: 27126
Валидационная: 9042
Тестовая: 9043

ВОПРОС 3 - Mutual Information анализ:
     Признак  MI_Score
6   poutcome      0.03
5      month      0.03
0        job      0.01
4    contact      0.01
3    housing      0.01
2  education      0.00
1    marital      0.00
Признак с наибольшей взаимной информацией: poutcome

ВОПРОС 4 - Точность базовой модели: 0.90

ВОПРОС 5 - Анализ важности признаков:
Без 'age': точность = 0.901, изменение = -0.000
Без 'balance': точность = 0.901, изменение = -0.000
Без 'marital': точность = 0.900, изменение = 0.001
Без 'previous': точность = 0.901, изменение = 0.000
Наименее важный признак: previous

ВОПРОС 6 - Подбор параметра регуляризации:
C = 0.01: точность = 0.897
C = 0.1: точность = 0.900
C = 1: точность = 0.901
C = 10: точн


ФИНАЛЬНЫЕ ОТВЕТЫ НА ВОПРОСЫ:
=
Вопрос 1: secondary
Вопрос 2: pdays и previous
Вопрос 3: poutcome
Вопрос 4: 0.9
Вопрос 5: previous
Вопрос 6: 10